In [1]:
# Installs the latest version of the Qiskit meta-package for circuit creation.
!pip install qiskit -U


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 4.4 MB/s eta 0:00:00


In [2]:
# Installs the latest version of the Qiskit Runtime package, which is needed to interact with the Qiskit Runtime primitives on IBM Cloud.
!pip install qiskit-ibm-runtime -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 10.6 MB/s eta 0:00:00
  Created wheel for ibm-platform-services: filename=ibm_platform_services-0.55.2-py3-none-any.whl size=339687 sha256=7cb15b9918921903419b5b839ae21230c9f5ec41997275b7a2e24b8ed42372d3
  Stored in directory: /root/.cache/pip/wheels/af/ef/67/6c17a948d54c37e26b0c927dc206d95a258e9a3f7ba7014c67
  Created wheel for ibm-cloud-sdk-core: f

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_cloud", token="", instance="")


In [4]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

# Run the sampler job locally using FakeManilaV2
backend = FakeManilaV2()

# Get ISA circuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_qc = pm.run(qc)

sampler = Sampler(backend=backend)
job = sampler.run([(isa_qc,)])
result = job.result()


<ipython-input-4-742fe92dc330>:19: DeprecationWarning: The 'backend' keyword arguments are deprecated as of qiskit-ibm-runtime 0.24.0 and will be removed no sooner than 3 months after the release date. Please use the 'mode' parameter instead.
  sampler = Sampler(backend=backend)
/usr/local/lib/python3.10/dist-packages/qiskit_ibm_runtime/fake_provider/fake_backend.py:374: RuntimeWarning: Aer not found, using qiskit.BasicSimulator and no noise.
  warnings.warn(


In [5]:
# Prepare the input circuit.
from qiskit import QuantumCircuit

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

# Execute the circuit
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeManilaV2

# Run the sampler job locally using FakeManilaV2
backend = FakeManilaV2()

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_bell = pm.run(bell)

sampler = Sampler(backend)

job = sampler.run([(isa_bell,)])
result = job.result()

pub_result = result[0]
# Get counts from the classical register "meas".
print(f" >> Counts for the meas output register: {pub_result.data.meas.get_counts()}")


 >> Counts for the meas output register: {'11': 517, '00': 507}


In [14]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_bloch_multivector

# Number of qubits in each layer
num_visible = 2
num_hidden = 2

# Create a quantum circuit with visible and hidden qubits
qc = QuantumCircuit(num_visible + num_hidden)

# Apply parameterized quantum gates (example: Rx gates)
for i in range(num_visible):
    for j in range(num_hidden):
        qc.cx(i, num_visible + j)  # CNOT gate to entangle visible and hidden qubits

# Visualize the circuit
print(qc.draw())


#New Simulate code
from qiskit import transpile

backend = FakeManilaV2()
new_circuit = transpile(qc, backend)
job = backend.run(new_circuit)
result = job.result()

# Get counts from the result
counts = result.get_counts(new_circuit)
print(f" >> Counts for the measurement output register: {counts}")

#pub_result = result
# Get counts from the classical register "meas".
#print(f" >> Counts for the meas output register: {pub_result.data.c.get_counts()}")


                         
q_0: ──■────■────────────
       │    │            
q_1: ──┼────┼────■────■──
     ┌─┴─┐  │  ┌─┴─┐  │  
q_2: ┤ X ├──┼──┤ X ├──┼──
     └───┘┌─┴─┐└───┘┌─┴─┐
q_3: ─────┤ X ├─────┤ X ├
          └───┘     └───┘
 >> Counts for the measurement output register: {}


In [18]:
from qiskit import QuantumCircuit, transpile

# Number of qubits in each layer
num_visible = 2
num_hidden = 2

# Create a quantum circuit with visible and hidden qubits and classical bits
qc = QuantumCircuit(num_visible + num_hidden, num_visible + num_hidden)

# Apply parameterized quantum gates (example: RX and RZ gates)
# Here, we use an angle parameter for the gates, which can be tuned.
for i in range(num_visible):
    qc.rx(1.0, i)  # RX gate with angle 1.0 on visible qubits
    qc.rz(1.0, i)  # RZ gate with angle 1.0 on visible qubits

# Apply entangling CNOT gates between visible and hidden qubits
for i in range(num_visible):
    for j in range(num_hidden):
        qc.cx(i, num_visible + j)

# Add measurement operations
qc.measure(range(num_visible + num_hidden), range(num_visible + num_hidden))

# Visualize the circuit
print(qc.draw())

# Backend configuration
backend = FakeManilaV2()  # Or use your specific backend
new_circuit = transpile(qc, backend)

# Run the circuit on the backend
job = backend.run(new_circuit)

# Get the results from the job
result = job.result()

# Get counts from the result
counts = result.get_counts()
print(f" >> Counts for the measurement output register: {counts}")


     ┌───────┐┌───────┐                    ┌─┐         
q_0: ┤ Rx(1) ├┤ Rz(1) ├──■────■────────────┤M├─────────
     ├───────┤├───────┤  │    │            └╥┘   ┌─┐   
q_1: ┤ Rx(1) ├┤ Rz(1) ├──┼────┼────■────■───╫────┤M├───
     └───────┘└───────┘┌─┴─┐  │  ┌─┴─┐  │   ║ ┌─┐└╥┘   
q_2: ──────────────────┤ X ├──┼──┤ X ├──┼───╫─┤M├─╫────
                       └───┘┌─┴─┐└───┘┌─┴─┐ ║ └╥┘ ║ ┌─┐
q_3: ───────────────────────┤ X ├─────┤ X ├─╫──╫──╫─┤M├
                            └───┘     └───┘ ║  ║  ║ └╥┘
c: 4/═══════════════════════════════════════╩══╩══╩══╩═
                                            0  2  1  3 
 >> Counts for the measurement output register: {'0000': 595, '1110': 189, '1101': 192, '0011': 48}
